# Budget Analysis

Explore categorised transaction data from `data/budget.db`.

In [ ]:
import sqlite3
import polars as pl
import matplotlib.pyplot as plt

conn = sqlite3.connect("data/budget.db")
df = pl.read_database("""
    SELECT date, raw_description, amount, currency, merchant_name,
           category, source, confidence, import_batch
    FROM transactions
    ORDER BY date DESC
""", conn)

df = df.with_columns([
    pl.col("date").str.strptime(pl.Date, "%Y-%m-%d"),
    pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").dt.strftime("%Y-%m").alias("month")
])
print(f"{len(df)} transactions loaded")
df.head(10)

## Spending by Category

In [ ]:
# Exclude Transfers to avoid double-counting credit card payments
spending = df.filter(
    (pl.col("amount") < 0) & (pl.col("category") != "Transfers")
).with_columns(
    pl.col("amount").abs()
)

by_cat = (
    spending
    .group_by("category")
    .agg(pl.col("amount").sum())
    .sort("amount", descending=True)
)
print(by_cat)
print(f"\nTotal spending: CHF {by_cat['amount'].sum():,.2f}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
by_cat_pd = by_cat.to_pandas().set_index("category")
by_cat_pd.plot.barh(ax=ax, legend=False)
ax.set_xlabel("CHF")
ax.set_title("Spending by Category")
ax.invert_yaxis()
plt.tight_layout()
plt.show()

## Monthly Spending Trend

In [ ]:
monthly = (
    spending
    .group_by("month")
    .agg(pl.col("amount").sum())
    .sort("month")
)

fig, ax = plt.subplots(figsize=(12, 5))
monthly_pd = monthly.to_pandas().set_index("month")
monthly_pd.plot.bar(ax=ax, legend=False)
ax.set_ylabel("CHF")
ax.set_title("Monthly Spending")
ax.tick_params(axis="x", rotation=45)
plt.tight_layout()
plt.show()

## Top Merchants

In [ ]:
top = (
    spending
    .group_by("merchant_name")
    .agg([
        pl.col("amount").sum().alias("total_chf"),
        pl.col("amount").count().alias("transactions")
    ])
    .sort("total_chf", descending=True)
    .head(20)
)
top

## Monthly Spending by Category

In [ ]:
pivot = (
    spending
    .group_by(["month", "category"])
    .agg(pl.col("amount").sum())
    .pivot(values="amount", index="month", columns="category")
    .fill_null(0)
    .sort("month")
)

fig, ax = plt.subplots(figsize=(14, 7))
pivot_pd = pivot.to_pandas().set_index("month")
pivot_pd.plot.bar(stacked=True, ax=ax)
ax.set_ylabel("CHF")
ax.set_title("Monthly Spending by Category")
ax.tick_params(axis="x", rotation=45)
ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

## Classification Quality

In [ ]:
print("Source distribution:")
source_counts = df.group_by("source").agg(pl.len().alias("count")).sort("count", descending=True)
print(source_counts)

mean_conf = df["confidence"].mean()
low_conf_count = df.filter(pl.col("confidence") < 0.8).height
uncat_count = df.filter(pl.col("category") == "Uncategorised").height

print(f"\nMean confidence: {mean_conf:.2%}")
print(f"Low confidence (<0.8): {low_conf_count} transactions")
print(f"Uncategorised: {uncat_count} transactions")

## Income vs Spending

In [ ]:
income = (
    df.filter(pl.col("amount") > 0)
    .group_by("month")
    .agg(pl.col("amount").sum())
    .sort("month")
)

# Exclude Transfers to avoid double-counting credit card payments
expenses = (
    df.filter((pl.col("amount") < 0) & (pl.col("category") != "Transfers"))
    .group_by("month")
    .agg(pl.col("amount").sum().abs())
    .sort("month")
)

summary = (
    income
    .join(expenses, on="month", how="outer", suffix="_expense")
    .fill_null(0)
    .rename({"amount": "Income", "amount_expense": "Spending"})
    .with_columns(
        (pl.col("Income") - pl.col("Spending")).alias("Net")
    )
    .sort("month")
)

fig, ax = plt.subplots(figsize=(12, 5))
summary_pd = summary.select(["month", "Income", "Spending"]).to_pandas().set_index("month")
summary_pd.plot.bar(ax=ax)
ax.axhline(y=0, color="black", linewidth=0.5)
ax.set_ylabel("CHF")
ax.set_title("Income vs Spending")
ax.tick_params(axis="x", rotation=45)
plt.tight_layout()
plt.show()

print(summary)